In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scopus/scopus.csv
/kaggle/input/papers-dataset/savedrecs.csv
/kaggle/input/newdata/scopus (3).csv


In [ ]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from huggingface_hub import login
login(token = "YOUR KEY")

In [ ]:
import torch
torch.cuda.empty_cache()  

In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 

model_name = "meta-llama/Llama-2-7b-chat-hf"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  
    low_cpu_mem_usage=True
).to("cuda")

# Load the CSV file
file_path = "/kaggle/input/scopus/scopus.csv"  # Update with your file path
df = pd.read_csv(file_path, encoding="utf-8") 

abstract_column = "Abstract"  


def is_relevant(abstract):
    if pd.isna(abstract) or not isinstance(abstract, str):
        return False  # Skip empty or invalid abstracts
    
    prompt = f"""
    You are a helpful AI that classifies research papers. 
    Determine whether the following abstract discusses the use of Generative AI in the retail industry and uses machine learning for ecommerce.
    Answer only 'yes' or 'no'. 

    Abstract: {abstract}
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=100)
    result = tokenizer.decode(output[0], skip_special_tokens=True).strip().lower()

    return "yes" in result


df["is_relevant"] = df[abstract_column].apply(is_relevant)
filtered_df = df[df["is_relevant"]]

output_path = "filtered_results.csv"
filtered_df.to_csv(output_path, index=False)
print(f"Filtered abstracts saved to {output_path}")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]